# Starwars Network Analysis
Given my background in the social sciences, I have found network analysis a very interesting discipline in data science especially as it relates to human social networks. With the evolution of big data, networks have become even more interesitng to study as we can witness them everywhere in our daily lives and can use statistical methods to study them. Studying the structure of a network allows us to answer questions about complex phenomena. Like whether a rumor is likely to spread across the network or who the most influenctial person in an organiation is. 


## Background
A network, or graph, is a representation of connections among a set of items where items are called nodes and the connections are called edges. 

Data retrived from Kaggle. The JSON can be downloaded from https://www.kaggle.com/datasets/ruchi798/star-wars.

The nodes in this network represent star wars characters and the edges represent conversations between them. Because a conversattion is a two way activity this is an undirected network where the edges have no direction and the relationship between the nodes is symetric. This network is weighted by the number of interactions between the characters, where more interactions corresponds to a stronger connection. 

This network is undirected as the node

DELETE ME. 

Interesting ideas: 
- Is a rumor likely to spread in this network? 
- Who are the most influential people in this organization? 

Questions:

Connectivity 
- Is the graph connected? A graph is connected if, for every pair of nodes, there is a path between them. 
- Is the graph strongly connected? A directed graph is strongly connected if, for every pair of nodes u and v there is a directed path from u and v and a directed path from v to u. 

Distance
The distance between two nodes: the length of the shortest path between them. 
- Are nodes far away or close to each other in this network> 
- Which nodes are closest and farthest to other nodes. 
- 

In [ ]:
# Standard library imports
import json
from typing import Tuple, Union

# Related third party imports
import altair as alt
import community as community_louvain
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

: 

## Functions

In [2]:
def get_character_interactions(weighted=False) -> Tuple[dict, list]:
    """
    Load the character interactions data from a JSON file and organize the nodes and edges information.

    Returns:
    - char_map (dict): A dictionary mapping character names to their corresponding index.
    - edges (list): A list of tuples representing the edges between characters.
    """
    # Load the data
    with open("./assets/starwars-full-interactions.json") as f:
        data = json.load(f)

    # Organize the nodes and edges information
    node_char_map = {entry["name"]: i for i, entry in enumerate(data["nodes"])}
    edges = [(edge["source"], edge["target"]) for edge in data["links"]]

    # If the graph is weighted, add the weights to the edges
    if weighted:
        weights = [edge["value"] for edge in data["links"]]
        edges = [(edge[0], edge[1], weight) for edge, weight in zip(edges, weights)]

    return node_char_map, edges

In [3]:
def get_graph(char_map, edges, weighted=False) -> nx.Graph:
    """
    Creates a networkx Graph object based on the given character map and edges.

    Parameters:
    - char_map (dict): A dictionary mapping character names to node IDs.
    - edges (list): A list of tuples representing the edges between characters.

    Returns:
    - nx.Graph: A networkx Graph object representing the character network.
    """
    # Initialize a Graph and add nodes and edges
    G = nx.Graph()
    G.add_nodes_from(char_map.values())
    G.add_edges_from(edges)

    # Add weights to the edges if the graph is weighted
    if weighted:
        for u, v, weight in edges:
            G[u][v]["weight"] = weight
    return G

In [4]:
def get_character_name(id, map=None) -> str | None:
    """
    Find the name of a character given their node index in the graph.

    Parameters:
    - id (int): The node index of the character.
    - map (dict): A dictionary mapping character names to their node indices.
                Default is char_map.

    Returns:
    - str or None: The name of the character if found, None otherwise.
    """
    # Get the character map if not provided, and find the character name based on node id
    if map is None:
        map, _ = get_character_interactions()
    for k, v in map.items():
        if v == id:
            return k
    return None

In [34]:
def print_network_summary(G):
    """
    Prints a summary of the network properties, interaction details, network structure insights,
    centrality measures, and path analysis of a given network.

    Parameters:
    - G (networkx.Graph): The network to analyze.

    Returns:
    - None
    """

    # Basic Network Properties
    summary = "=== Network Summary ===\n"
    summary += f"{'Number of characters:':<40} {len(G.nodes)}\n"
    summary += f"{'Number of interactions:':<40} {len(G.edges)}\n"
    summary += f"{'Number of connected components:':<40} {nx.number_connected_components(G)}\n"
    summary += f"{'Number of isolated nodes:':<40} {len(list(nx.isolates(G)))}\n\n"

    # Interaction Details
    degrees = dict(G.degree())
    max_degree_node = max(degrees, key=degrees.get)
    min_degree_node = min(degrees, key=degrees.get)
    summary += "=== Interaction Details ===\n"
    summary += f"{'Character with the most interactions:':<40} {get_character_name(max_degree_node)} ({degrees[max_degree_node]} interactions)\n"
    summary += f"{'Character with the least interactions:':<40} {get_character_name(min_degree_node)} ({degrees[min_degree_node]} interactions)\n"
    summary += f"{'Average number of interactions:':<40} {np.mean(list(degrees.values())):.2f}\n"
    summary += f"{'Standard deviation of interactions:':<40} {np.std(list(degrees.values())):.2f}\n\n"

    # Network Structure Insights
    summary += "=== Network Structure Insights ===\n"
    summary += f"{'Network density:':<40} {nx.density(G):.4f}\n"
    summary += f"{'Average clustering coefficient:':<40} {nx.average_clustering(G):.4f}\n\n"

    # Centrality Measures
    betweenness = nx.betweenness_centrality(G)
    closeness = nx.closeness_centrality(G)
    most_central_node = max(betweenness, key=betweenness.get)
    least_central_node = min(betweenness, key=betweenness.get)
    closest_node = max(closeness, key=closeness.get)
    furthest_node = min(closeness, key=closeness.get)
    summary += "=== Centrality Measures ===\n"
    summary += f"{'Character with highest betweenness centrality:':<40} {get_character_name(most_central_node)} ({most_central_node})\n"
    summary += f"{'Character with lowest betweenness centrality:':<40} {get_character_name(least_central_node)} ({least_central_node})\n"
    summary += f"{'Character with highest closeness centrality:':<40} {get_character_name(closest_node)} ({closest_node})\n"
    summary += f"{'Character with lowest closeness centrality:':<40} {get_character_name(furthest_node)} ({furthest_node})\n\n"

    # Path Analysis
    if nx.is_connected(G):
        summary += "=== Path Analysis ===\n"
        summary += f"{'Average shortest path length:':<40} {nx.average_shortest_path_length(G):.4f}\n"
        summary += f"{'Network diameter:':<40} {nx.diameter(G)}"
    else:
        summary += "Network is not connected; average shortest path length and diameter are not defined."

    print(summary)

In [12]:
def get_edge_trace(G):
    """
    Generate the edge trace for a network graph.

    Parameters:
    - G (networkx.Graph): The input graph.

    Returns:
    - edge_x (list): The x-coordinates of the edges.
    - edge_y (list): The y-coordinates of the edges.
    - line_widths (list): The scaled weights for the edges. Defaults to 0.5 (1 * 0.5) if no weight is provided.
    """

    # Initialize lists to hold the coordinates and scaled weights for edges
    edge_x = []
    edge_y = []
    line_widths = []

    # Iterate over the edges to prepare the edge trace
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]["pos"]
        x1, y1 = G.nodes[edge[1]]["pos"]
        edge_x.extend([x0, x1, None])  # Add x-coordinates and a None to break the line
        edge_y.extend([y0, y1, None])  # Add y-coordinates and a None to break the line

        # If the graph is weighted, scale the edge weight for line width
        line_widths.append(
            edge[2].get("weight", 1) * 0.5
        )  # Scale edge weight for line width

    return edge_x, edge_y, line_widths


def get_node_trace(G):
    """
    Generate the node trace for a network graph.

    Parameters:
    - G (networkx.Graph): The network graph.

    Returns:
    - node_x (list): The x-coordinates of the nodes.
    - node_y (list): The y-coordinates of the nodes.
    - text (list): The text labels for the nodes.
    """

    # Initialize lists to hold the coordinates and text for nodes
    node_x = []
    node_y = []
    text = []

    # Iterate over the nodes to prepare the node trace
    for node in G.nodes():
        x, y = G.nodes[node]["pos"]
        node_x.append(x) # Add x-coordinate
        node_y.append(y) # Add y-coordinate
        text.append(f"{node}: {get_character_name(node)}") # Add character name

    return node_x, node_y, text

In [13]:
def plot_network(G):
    """
    Plots the network graph of a given graph G. This includes the network structure, node connections, degree and rank plot, and a degree histogram.

    Parameters:
    - G (networkx.Graph): The graph to be plotted.

    Returns:
    - None
    """

    # Position nodes using the spring layout
    pos = nx.spring_layout(G)
    nx.set_node_attributes(G, pos, 'pos')

    edge_x, edge_y, line_widths = get_edge_trace(G)
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
        mode="lines"
    )

    node_x, node_y, text = get_node_trace(G)
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        hoverinfo="text",
        text=text,
        marker=dict(
            showscale=True,  # Ensure the color scale is shown
            colorscale="YlGnBu",
            reversescale=True,
            color=[],  # This will be set based on the degree of the nodes
            size=10,
            colorbar=dict(
                thickness=15,
                title="Node Connections",
                xanchor="left",
                titleside="right"
            ),
            line_width=2
        )
    )

    # Assign node colors based on the degree of the nodes
    node_adjacencies = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
    node_trace.marker.color = node_adjacencies  # Set colors based on connections

    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "scatter", "colspan": 2}, None],
               [{"type": "scatter"}, {"type": "bar"}]],
        subplot_titles=("Network", "Degree Rank", "Degree Histogram"),
        vertical_spacing=0.1
    )

    fig.add_trace(edge_trace, row=1, col=1)
    fig.add_trace(node_trace, row=1, col=1)

    degrees = [val for (node, val) in G.degree()]
    degree_counts = [degrees.count(i) for i in range(max(degrees)+1)]

    fig.add_trace(
        go.Scatter(x=list(range(len(degrees))), y=sorted(degrees, reverse=True), mode="lines+markers"),
        row=2, col=1
    )
    fig.add_trace(go.Histogram(x=degree_counts), row=2, col=2)

    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Star Wars Character Interactions and Degree Analysis"
    )

    fig.update_xaxes(title_text="Rank", row=2, col=1)
    fig.update_yaxes(title_text="Degree", row=2, col=1)
    fig.update_xaxes(title_text="Degree", row=2, col=2)
    fig.update_yaxes(title_text="Count", row=2, col=2)

    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)

    fig.show()

In [35]:
print_network_summary(G)

=== Network Summary ===
Number of characters:                    109
Number of interactions:                  398
Number of connected components:          1
Number of isolated nodes:                0

=== Interaction Details ===
Character with the most interactions:    ANAKIN (41 interactions)
Character with the least interactions:   TARPALS (1 interactions)
Average number of interactions:          7.30
Standard deviation of interactions:      7.73

=== Network Structure Insights ===
Network density:                         0.0676
Average clustering coefficient:          0.6830

=== Centrality Measures ===
Character with highest betweenness centrality:OBI-WAN (4)
Character with lowest betweenness centrality: PK-4(2)
Character with highest closeness centrality: OBI-WAN(4)
Character with lowest closeness centrality: COLONEL DATOO(106)

=== Path Analysis ===
Average shortest path length:            2.7278
Network diameter:                        6


## Code

In [40]:
char_map, edges = get_character_interactions()
G = get_graph(char_map, edges)
print_network_summary(G)

plot_network(G)

=== Network Summary ===
Number of characters:                    110
Number of interactions:                  398
Number of connected components:          2
Number of isolated nodes:                1

=== Interaction Details ===
Character with the most interactions:    ANAKIN (41 interactions)
Character with the least interactions:   GOLD FIVE (0 interactions)
Average number of interactions:          7.24
Standard deviation of interactions:      7.73

=== Network Structure Insights ===
Network density:                         0.0664
Average clustering coefficient:          0.6768

=== Centrality Measures ===
Character with highest betweenness centrality:OBI-WAN (4)
Character with lowest betweenness centrality: PK-4(2)
Character with highest closeness centrality: OBI-WAN(4)
Character with lowest closeness centrality: GOLD FIVE(76)

Network is not connected; average shortest path length and diameter are not defined.


1.	Connected Components of G (Top Plot):
- This is a visualization of the network (graph) itself.
- Nodes (blue dots) represent entities or points.
- Edges (lines connecting the nodes) represent relationships or connections between the nodes.
- The visualization helps to identify how nodes are connected to each other and to observe the structure of the network. Clusters or densely connected groups can be identified.
2.	Degree Rank Plot (Bottom Left):
- This plot shows the degree of each node (number of connections a node has) versus the rank of the node when nodes are ordered by degree.
- The x-axis represents the rank of nodes when sorted by their degree, and the y-axis represents the degree of the nodes.
- A steep drop indicates that a few nodes have high degrees (many connections), and most nodes have fewer connections. This can indicate the presence of hub nodes in the network.
3.	Degree Histogram (Bottom Right):
- This histogram displays the distribution of node degrees in the network.
- The x-axis represents the degree (number of connections) and the y-axis represents the number of nodes with that degree.
- It helps to understand the overall distribution of connections in the network. A right-skewed distribution, as seen here, suggests that most nodes have a low degree, while a few nodes have a high degree.


# Remove Isolated Node

In [19]:
G.remove_nodes_from(list(nx.isolates(G)))
assert 76 not in G.nodes